In [1]:
import pandas as pd 
import os 
pd.options.display.max_columns = 50

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer
#import re
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer

In [2]:
os.chdir(r'C:\Users\saadb\recommendation_project\jupyter_notebook')

In [3]:
df = pd.read_csv('datasets\complete_dataset_.csv',parse_dates=['release_date'])

In [4]:
df.head(3)

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast_names,cast_main_name,cast_main_gender,Director,Writers,Producer,keywords
0,False,Toy Story Collection,30.0,Animation|Comedy|Family,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373.554033,81.0,English,Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks|Tim Allen|Don Rickles|Jim Varney,Tom Hanks,M,NaN,NaN,NaN,jealousy|toy|boy|friendship|friends|rivalry|bo...
1,False,NaN,65.0,Adventure|Fantasy|Family,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262.797249,104.0,English|Français,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Robin Williams,M,Joe Johnston,Jonathan Hensleigh|Chris van Allsburg|Greg Tay...,William Teitler,NaN
2,False,Grumpy Old Men Collection,NaN,Romance|Comedy,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,NaN,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,Walter Matthau,M,Howard Deutch,Mark Steven Johnson|Mark Steven Johnson,NaN,fishing|best friend|duringcreditsstinger|old men


Using Binary Feature Matrix Vectorizer

In [5]:
genre_matrix = pd.get_dummies(df['genres'].str.split("|").apply(pd.Series).stack()).sum(level=0)
genre_matrix

C:\Users\saadb\AppData\Local\Temp\ipykernel_15940\1818501217.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  genre_matrix = pd.get_dummies(df['genres'].str.split("|").apply(pd.Series).stack()).sum(level=0)


,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,missing
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46420,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
46421,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46422,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
46423,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [24]:
similarity = cosine_similarity(genre_matrix)

In [59]:
def get_recommendations(title, top_n=5):
    # Find the index of the movie with the given title
    idx = df[df['title'] == title].index[0]
    
    # Get the cosine similarity scores for the movie
    similarity_scores = list(enumerate(similarity[idx]))
    
    # Sort the similarity scores in descending order
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top_n movie indices
    movie_indices = [i[0] for i in similarity_scores[1:top_n+1]]
    
    # Return the top_n most similar movies
    return df['title'].iloc[movie_indices]

In [63]:
#title = input("Enter the title of your favorite movie: ")
# Get the recommended movies
print("Top 5 similar movies:")
print(get_recommendations('Jumanji'))

Top 5 similar movies:
59                    The Indian in the Cupboard
898                             The Wizard of Oz
1866                                   Labyrinth
1992                                Return to Oz
4783    Harry Potter and the Philosopher's Stone
Name: title, dtype: object


Using Bags Of Words

In [5]:
title = df['title'].to_list()
title[0]

'Toy Story'

In [6]:
genres = df['genres'].str.split('|').to_list()
genres[0]

['Animation', 'Comedy', 'Family']

bow will be in the follwoing form 

title as index and all genres as columns in cells will be 1 if the movies titles has column genres and thats it 

In [7]:
#so to make it in this format we need some sort of dictunery that store key genres and values as 1 and index will be movies 
#title then fill the rest of cells with 0

def create_bow(genre_list):
    bow = {}
    for genre in genre_list:
        bow[genre] = 1
    return bow


In [8]:
bag_of_words = [create_bow(genre) for genre in genres] 

In [9]:
bag_of_words

[{'Animation': 1, 'Comedy': 1, 'Family': 1},
 {'Adventure': 1, 'Fantasy': 1, 'Family': 1},
 {'Romance': 1, 'Comedy': 1},
 {'Comedy': 1, 'Drama': 1, 'Romance': 1},
 {'Comedy': 1},
 {'Action': 1, 'Crime': 1, 'Drama': 1, 'Thriller': 1},
 {'Comedy': 1, 'Romance': 1},
 {'Action': 1, 'Adventure': 1, 'Drama': 1, 'Family': 1},
 {'Action': 1, 'Adventure': 1, 'Thriller': 1},
 {'Adventure': 1, 'Action': 1, 'Thriller': 1},
 {'Comedy': 1, 'Drama': 1, 'Romance': 1},
 {'Comedy': 1, 'Horror': 1},
 {'Family': 1, 'Animation': 1, 'Adventure': 1},
 {'History': 1, 'Drama': 1},
 {'Action': 1, 'Adventure': 1},
 {'Drama': 1, 'Crime': 1},
 {'Drama': 1, 'Romance': 1},
 {'Crime': 1, 'Comedy': 1},
 {'Crime': 1, 'Comedy': 1, 'Adventure': 1},
 {'Action': 1, 'Comedy': 1, 'Crime': 1},
 {'Comedy': 1, 'Thriller': 1, 'Crime': 1},
 {'Drama': 1, 'Thriller': 1},
 {'Action': 1, 'Adventure': 1, 'Crime': 1, 'Thriller': 1},
 {'Drama': 1, 'Fantasy': 1, 'Science Fiction': 1, 'Thriller': 1},
 {'Drama': 1, 'Romance': 1},
 {'Drama'

In [12]:
genre_df = pd.DataFrame(bag_of_words , index=title).fillna(0)

In [13]:
genre_df

,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,History,Science Fiction,Mystery,War,Foreign,missing,Music,Documentary,Western,TV Movie
Toy Story,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Waiting to Exhale,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Subdue,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Century of Birthing,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Betrayal,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Satan Triumphant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
cosine_similarity = cosine_similarity(genre_df)

In [15]:
cosine_similarity

array([[1.        , 0.33333333, 0.40824829, ..., 0.        , 0.        ,
        0.        ],
       [0.33333333, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.40824829, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        1.        ]])

In [16]:
similarity_df = pd.DataFrame(cosine_similarity, index=genre_df.index, columns=genre_df.index)

In [17]:
similarity_df

,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,The American President,Dracula: Dead and Loving It,Balto,Nixon,Cutthroat Island,Casino,Sense and Sensibility,Four Rooms,Ace Ventura: When Nature Calls,Money Train,Get Shorty,Copycat,Assassins,Powder,Leaving Las Vegas,...,Cadet Kelly,The Scheming Gambler's Paradise,The Man with the Rubber Head,The Living Playing Cards,The Hilarious Posters,The Devilish Tenant,The Untameable Whiskers,The Imperceptable Transmutations,Pooh's Heffalump Halloween Movie,The One-Man Band,The Fat and Lean Wrestling Match,Deep Hearts,Mom,The Morning After,St. Michael Had a Rooster,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,Robin Hood,Subdue,Century of Birthing,Betrayal,Satan Triumphant,Queerama
Toy Story,1.000000,0.333333,0.408248,0.333333,0.577350,0.000000,0.408248,0.288675,0.000000,0.000000,0.333333,0.408248,0.666667,0.000000,0.000000,0.000000,0.000000,0.408248,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,...,0.577350,0.0,0.333333,0.00000,0.408248,0.408248,0.0,0.0,0.816497,0.000000,0.408248,0.0,0.000000,0.408248,0.0,0.000000,0.0,0.0,0.0,0.000000,0.408248,0.000000,0.000000,0.0,0.0
Jumanji,0.333333,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.577350,0.333333,0.333333,0.000000,0.000000,0.666667,0.000000,0.408248,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.288675,0.288675,0.000000,...,0.000000,0.0,0.333333,0.57735,0.408248,0.408248,0.0,0.0,0.408248,0.333333,0.408248,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.408248,0.000000,0.000000,0.0,0.0
Grumpier Old Men,0.408248,0.000000,1.000000,0.816497,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000,0.816497,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.408248,0.408248,0.408248,0.000000,0.000000,0.000000,0.500000,...,0.707107,0.0,0.408248,0.00000,0.500000,0.500000,0.0,0.0,0.000000,0.000000,0.500000,0.0,0.000000,0.500000,0.0,0.000000,0.0,0.0,0.0,0.408248,0.000000,0.000000,0.000000,0.0,0.0
Waiting to Exhale,0.333333,0.000000,0.816497,1.000000,0.577350,0.288675,0.816497,0.288675,0.000000,0.000000,1.000000,0.408248,0.000000,0.408248,0.000000,0.408248,0.816497,0.408248,0.333333,0.333333,0.333333,0.408248,0.000000,0.288675,0.816497,...,0.577350,0.0,0.333333,0.00000,0.408248,0.408248,0.0,0.0,0.000000,0.000000,0.408248,0.0,0.333333,0.816497,0.0,0.000000,0.0,0.0,0.0,0.666667,0.408248,0.577350,0.333333,0.0,0.0
Father of the Bride Part II,0.577350,0.000000,0.707107,0.577350,1.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.577350,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.577350,0.577350,0.577350,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.0,0.577350,0.00000,0.707107,0.707107,0.0,0.0,0.000000,0.000000,0.707107,0.0,0.000000,0.707107,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Subdue,0.408248,0.408248,0.000000,0.408248,0.000000,0.353553,0.000000,0.707107,0.000000,0.000000,0.408248,0.000000,0.408248,0.500000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.353553,0.500000,...,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.500000,0.000000,0.000000,0.0,0.408248,0.500000,0.0,0.000000,0.0,0.0,0.0,0.408248,1.000000,0.707107,0.408248,0.0,0.0
Century of Birthing,0.000000,0.000000,0.000000,0.577350,0.000000,0.500000,0.000000,0.500000,0.000000,0.000000,0.577350,0.000000,0.000000,0.707107,0.000000,0.707107,0.707107,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.500000,0.707107,...,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.577350,0.707107,0.0,0.000000,0.0,0.0,0.0,0.577350,0.707107,1.000000,0.577350,0.0,0.0
Betrayal,0.000000,0

In [27]:
movie_index = similarity_df.index.get_loc('Waiting to Exhale')

In [28]:
top_10 = similarity_df.iloc[movie_index].sort_values(ascending=False)[1:20]


In [29]:
print(f'Top 10 similar movies to Toy Story:')
print(top_10)

Top 10 similar movies to Toy Story:
He Was a Quiet Man                        1.0
Quartet                                   1.0
Unmade Beds                               1.0
I Hate Luv Storys                         1.0
Mere Brother Ki Dulhan                    1.0
Jet Lag                                   1.0
The Man Who Loved Women                   1.0
Semi-Tough                                1.0
The Catered Affair                        1.0
The Spectacular Now                       1.0
If I Were You                             1.0
Love in the Afternoon                     1.0
The Lifeguard                             1.0
The Cyclone                               1.0
Sex and Breakfast                         1.0
Crazy Kind of Love                        1.0
The Irony of Fate, or Enjoy Your Bath!    1.0
Café Society                              1.0
Brief Interviews with Hideous Men         1.0
Name: Waiting to Exhale, dtype: float64


Using Tf-IDF Vectorizer

In [5]:
genres_combined = df['genres'].str.replace('|',' ')

C:\Users\saadb\AppData\Local\Temp\ipykernel_19884\2538760687.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  genres_combined = df['genres'].str.replace('|',' ')


In [8]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(genres_combined)

In [10]:
cosine_similarity = cosine_similarity(tfidf_matrix)

In [12]:
similarity_df = pd.DataFrame(cosine_similarity, index=df['title'], columns=df['title'])

In [13]:
similarity_df

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,The American President,Dracula: Dead and Loving It,Balto,Nixon,Cutthroat Island,Casino,Sense and Sensibility,Four Rooms,Ace Ventura: When Nature Calls,Money Train,Get Shorty,Copycat,Assassins,Powder,Leaving Las Vegas,...,Cadet Kelly,The Scheming Gambler's Paradise,The Man with the Rubber Head,The Living Playing Cards,The Hilarious Posters,The Devilish Tenant,The Untameable Whiskers,The Imperceptable Transmutations,Pooh's Heffalump Halloween Movie,The One-Man Band,The Fat and Lean Wrestling Match,Deep Hearts,Mom,The Morning After,St. Michael Had a Rooster,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,Robin Hood,Subdue,Century of Birthing,Betrayal,Satan Triumphant,Queerama
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.363755,0.225670,0.202450,0.369655,0.000000,0.225670,0.381611,0.000000,0.000000,0.202450,0.208502,0.784901,0.000000,0.000000,0.000000,0.000000,0.204965,0.153641,0.165839,0.168680,0.000000,0.000000,0.000000,0.000000,...,0.369655,0.0,0.119262,0.000000,0.181469,0.181469,0.0,0.0,0.929169,0.000000,0.181469,0.0,0.000000,0.287708,0.0,0.000000,0.0,0.0,0.0,0.000000,0.566567,0.000000,0.000000,0.0,0.0
Jumanji,0.363755,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.663351,0.359755,0.359755,0.000000,0.000000,0.621913,0.000000,0.420239,0.000000,0.000000,0.000000,0.360162,0.000000,0.000000,0.000000,0.305331,0.327332,0.000000,...,0.000000,0.0,0.347185,0.606373,0.528277,0.528277,0.0,0.0,0.391484,0.424866,0.528277,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.523695,0.000000,0.000000,0.0,0.0
Grumpier Old Men,0.225670,0.000000,1.000000,0.897107,0.610487,0.000000,1.000000,0.000000,0.000000,0.000000,0.897107,0.344341,0.000000,0.000000,0.000000,0.000000,0.672601,0.338501,0.253738,0.273884,0.278575,0.000000,0.000000,0.000000,0.672601,...,0.610487,0.0,0.196962,0.000000,0.299697,0.299697,0.0,0.0,0.000000,0.000000,0.299697,0.0,0.000000,0.475152,0.0,0.000000,0.0,0.0,0.0,0.510608,0.000000,0.000000,0.000000,0.0,0.0
Waiting to Exhale,0.202450,0.000000,0.897107,1.000000,0.547673,0.143497,0.897107,0.127759,0.000000,0.000000,1.000000,0.308911,0.000000,0.165209,0.000000,0.209171,0.836693,0.303672,0.227631,0.245703,0.249912,0.240258,0.000000,0.108415,0.836693,...,0.547673,0.0,0.176696,0.000000,0.268861,0.268861,0.0,0.0,0.000000,0.000000,0.268861,0.0,0.168894,0.703665,0.0,0.000000,0.0,0.0,0.0,0.635179,0.189680,0.441813,0.180097,0.0,0.0
Father of the Bride Part II,0.369655,0.000000,0.610487,0.547673,1.000000,0.000000,0.610487,0.000000,0.000000,0.000000,0.547673,0.564043,0.000000,0.000000,0.000000,0.000000,0.000000,0.554477,0.415632,0.448632,0.456316,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.0,0.322630,0.000000,0.490915,0.490915,0.0,0.0,0.000000,0.000000,0.490915,0.0,0.000000,0.778315,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Subdue,0.566567,0.523695,0.000000,0.189680,0.000000,0.139441,0.000000,0.673549,0.000000,0.000000,0.189680,0.000000,0.515082,0.160539,0.000000,0.203258,0.226702,0.000000,0.000000,0.000000,0.000000,0.233466,0.000000,0.105350,0.226702,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.609756,0.000000,0.000000,0.0,0.164120,0.269560,0.0,0.000000,0.0,0.0,0.0,0.172102,1.000000,0.429323,0.175006,0.0,0.0
Century of Birthing,0.000000,0.000000,0.000000,0.441813,0.000000,0.324792,0.000000,0.289170,0.000000,0.000000,0.441813,0.000000,0.000000,0.373935,0.000000,0.473439,0.528046,0.000000,0.000000,0.000000,0.000000,0.543800,0.000000,0.245386,0.528046,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.382275,0.627874,0.0,0.000000,0.0,0.

In [14]:
movie_index = similarity_df.index.get_loc('Jumanji')

In [18]:
top_10 = similarity_df.iloc[movie_index].sort_values(ascending=False)[1:25]

In [19]:
print(f'Top 10 similar movies to Jumanji:')
print(top_10)

Top 10 similar movies to Jumanji:
title
The Adventures of Buratino                                  1.0
Escape to Witch Mountain                                    1.0
The Sword and the Dragon                                    1.0
Pan                                                         1.0
The Water Horse                                             1.0
The Fairy King Of Ar                                        1.0
The Brothers Lionheart                                      1.0
Return to Oz                                                1.0
Mama                                                        1.0
Labyrinth                                                   1.0
Gulliver's Travels                                          1.0
Percy Jackson: Sea of Monsters                              1.0
Jack and the Beanstalk                                      1.0
Pete's Dragon                                               1.0
Sapphire Blue                                               1.0


final model 

In [6]:
df.head(5)

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast_names,cast_main_name,cast_main_gender,Director,Writers,Producer,keywords
0,False,Toy Story Collection,30.0,Animation|Comedy|Family,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373.554033,81.0,English,Released,NaN,Toy Story,False,7.7,5415.0,Tom Hanks|Tim Allen|Don Rickles|Jim Varney,Tom Hanks,M,NaN,NaN,NaN,jealousy|toy|boy|friendship|friends|rivalry|bo...
1,False,NaN,65.0,Adventure|Fantasy|Family,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262.797249,104.0,English|Français,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Robin Williams,M,Joe Johnston,Jonathan Hensleigh|Chris van Allsburg|Greg Tay...,William Teitler,NaN
2,False,Grumpy Old Men Collection,NaN,Romance|Comedy,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,NaN,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,Walter Matthau,M,Howard Deutch,Mark Steven Johnson|Mark Steven Johnson,NaN,fishing|best friend|duringcreditsstinger|old men
3,False,NaN,16.0,Comedy|Drama|Romance,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81.452156,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,Forest Whitaker,Ronald Bass|Terry McMillan|Terry McMillan,Caron K,based on novel|interracial relationship|single...
4,False,Father of the Bride Collection,NaN,Comedy,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76.578911,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Steve Martin,M,Charles Shyer,Nancy Meyers|Albert Hackett,Nancy Meyers,baby|midlife crisis|confidence|aging|daughter|...


In [7]:
df['content'] = df['adult'].astype(str) + ' ' + df['belongs_to_collection'].astype(str) + ' ' + df['budget'].astype(str) + ' ' + df['genres'] +  ' ' + df['original_language'] + ' ' + df['overview'] + ' ' + df['popularity'].astype(str) + ' ' +  df['production_companies'].astype(str) + ' ' + df['production_countries'].astype(str) + ' ' + df['release_date'].astype(str) + ' ' + df['revenue'].astype(str) + ' ' + df['runtime'].astype(str) + ' ' + df['spoken_languages'].astype(str) + ' ' + df['tagline'].astype(str) + ' ' + df['vote_average'].astype(str) + ' ' + df['vote_count'].astype(str) + ' ' + df['cast_names'].astype(str) + ' ' + df['cast_main_name'].astype(str) + ' ' + df['cast_main_gender'].astype(str) + ' ' + df['Director'].astype(str) + ' ' + df['Writers'].astype(str) + df['Producer'].astype(str)
df['content'] = df['content'].fillna('')
df['content'].head(5)

0    False Toy Story Collection 30.0 Animation|Come...
1    False nan 65.0 Adventure|Fantasy|Family en Whe...
2    False Grumpy Old Men Collection nan Romance|Co...
3    False nan 16.0 Comedy|Drama|Romance en Cheated...
4    False Father of the Bride Collection nan Comed...
Name: content, dtype: object

In [8]:
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(df['content'])

In [9]:
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(bow)


In [10]:
lsa = TruncatedSVD(n_components=100, algorithm='arpack')
lsa.fit(tfidf)

TruncatedSVD(algorithm='arpack', n_components=100)

In [11]:
movie_index = df[df['title'] == 'Toy Story'].index[0]

In [12]:
similarity_scores = cosine_similarity(tfidf[movie_index], tfidf)

In [13]:
similar_movies = list(enumerate(similarity_scores[0]))
sorted_similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:20]


In [14]:
for i, score in sorted_similar_movies:
    print("{}: {}".format(i, df.loc[i, 'title']))

3008: Toy Story 2
15411: Toy Story 3
41943: Crisis in Six Scenes
21979: Toy Story of Terror!
5234: Hollywood Ending
3507: Small Time Crooks
2185: Stardust Memories
1054: Sleeper
1862: Take the Money and Run
25845: Toy Story That Time Forgot
1055: Bananas
1208: Manhattan
6960: Play It Again, Sam
5990: A Midsummer Night's Sex Comedy
2646: Radio Days
11533: Don't Drink the Water
3703: Love and Death
24570: Small Fry
8191: The Man with One Red Shoe


In [15]:
movie_index

0

In [16]:
similarity_scores

array([[1.        , 0.03789689, 0.02739553, ..., 0.01312559, 0.03197843,
        0.02396105]])

In [17]:
similar_movies

[(0, 1.0),
 (1, 0.037896890096474276),
 (2, 0.02739553243215081),
 (3, 0.036136852691643746),
 (4, 0.04793346472534976),
 (5, 0.028694795868088326),
 (6, 0.022085261990169865),
 (7, 0.06929065033064816),
 (8, 0.01942180296979791),
 (9, 0.022729250527803023),
 (10, 0.02664490936824504),
 (11, 0.03616503493498452),
 (12, 0.04086715238843103),
 (13, 0.0330364435572232),
 (14, 0.028993220645541103),
 (15, 0.023249074563611374),
 (16, 0.04527048546072),
 (17, 0.06258071671791607),
 (18, 0.04047515058969911),
 (19, 0.07191014424839676),
 (20, 0.04615651133864041),
 (21, 0.03276983396524724),
 (22, 0.037247180967616325),
 (23, 0.034024207886617926),
 (24, 0.03170977007996636),
 (25, 0.02314888380412635),
 (26, 0.03701766714872914),
 (27, 0.02217989204978564),
 (28, 0.038711457738199305),
 (29, 0.03683970141247869),
 (30, 0.030360242480603556),
 (31, 0.0368111290455002),
 (32, 0.0),
 (33, 0.05078398904477793),
 (34, 0.01727571274913722),
 (35, 0.034668014775156665),
 (36, 0.03799995356567734),

<1x204995 sparse matrix of type '<class 'numpy.float64'>'
	with 73 stored elements in Compressed Sparse Row format>